In [2]:
import coremltools
from PIL import Image
import os
import re
import shutil
import json
import yaml
import csv
from collections import OrderedDict

## Open source model

In [2]:
model_folder = '/Users/landyrev/Downloads/latest'

model_name = filter(
    lambda x: re.match(r'.*.caffemodel', x, re.IGNORECASE), 
    os.listdir(model_folder)
)[0]

model_path = os.path.join(model_folder, model_name)

OSError: [Errno 2] No such file or directory: '/Users/landyrev/Downloads/latest'

## Load and convert model

In [3]:
m = coremltools.converters.caffe.convert(
    (
        model_path, os.path.join(model_folder, 'deploy.prototxt'), 
        os.path.join(model_folder, 'mean.binaryproto')
    ),
    class_labels=os.path.join(model_folder, 'labels.txt'), 
    image_input_names='data'
)

## Add model metadata

In [4]:
m.author = 'Aleksey Landyrev'
m.short_description = 'How to draw model'
m.save('realmodel.mlmodel')

## Copy model into target project

In [5]:
shutil.copy('./realmodel.mlmodel', '../../htd/Data/realmodel.mlmodel')

## Copy needed images

In [12]:
android_bundle = '../../../htd-android/app/src/main/assets/Images.bundle'
ios_bundle = '../../htd/Data/Images.bundle'

shutil.rmtree(android_bundle)
os.mkdir(android_bundle)
shutil.rmtree(ios_bundle)
os.mkdir(ios_bundle)

for label in open(os.path.join(".", 'labels.txt')):
    label = label[:-1].replace(" ", "_")
    if label == "random":
        continue
    
    src_dir = os.path.join('../AllImages/', label)
    android_dst_dir = os.path.join(android_bundle, label)
    ios_dst_dir = os.path.join(ios_bundle, label)
    meta_file = os.path.join(src_dir, 'meta.yaml')
    
    os.mkdir(android_dst_dir)
    os.mkdir(ios_dst_dir)
    
    for image in os.listdir(src_dir):
        if not re.match(r'.*(\.png|\.jpg)', image, re.IGNORECASE):
            continue
        shutil.copy(
            os.path.join(src_dir, image),
            os.path.join(ios_dst_dir, label + "_" + image)
        )
        shutil.copy(
            os.path.join(src_dir, image),
            os.path.join(android_dst_dir, label + "_" + image)
        )

# Load meta

In [13]:
with open('./levels.csv') as f:
    reader = csv.DictReader(f)
    tutorials = [t for t in reader if not t["name"] == "random"]
    
for tutorial in tutorials:
    label = tutorial["name"]
    dst_dir = os.path.join("../../htd/Data/Images.bundle/", label)
    tutorial.update({
        "tutorials": sorted([os.path.join(label, x) 
             for x in os.listdir(dst_dir)
             if x.endswith('.jpg')
        ])
    })

yaml.safe_dump(
    tutorials,
    open(os.path.join(android_bundle, "levels.yaml"), "w"),
    default_flow_style=False,
    allow_unicode=True
)

yaml.safe_dump(
    tutorials,
    open(os.path.join(ios_bundle, "levels.yaml"), "w"),
    default_flow_style=False,
    allow_unicode=True
)

with open('../../htd/Data/ru.lproj/Levels.strings', 'w') as f:
    f.writelines(
        ['"{0}" = "{1}";\n'.format(t["name"], t["title_ru"]) for t in tutorials]
    )
    
with open('../../htd/Data/en.lproj/Levels.strings', 'w') as f:
    f.writelines(
        ['"{0}" = "{1}";\n'.format(t["name"], t["title_en"]) for t in tutorials]
    )

## Try to predict test image

In [13]:
def predict(image_name):
    image = Image.open(image_name).convert('L')
    image = image.resize((224, 224))
    features = m.predict({
        'data': image
    }).get('softmax')
    for feature in features:
        print feature, int(features[feature] * 100)
        
predict('/Users/landyrev/Downloads/fvsdfvsdfv.jpg')

lutz_swan0 0
lutz_lighthouse0 0
random 0
lutz_boat0 99
lutz_pug0 0
lutz_woman0 0
